In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from explicit import waiter, XPATH
import time
import pandas as pd
from bs4 import BeautifulSoup

In [26]:
username = "owasp.viit"
password = "owasp@vignansiit"

In [27]:
def login(driver):
    driver.get("https://www.instagram.com/accounts/login/")
    # entering the username in username field
    waiter.find_write(driver, "//*[@id='loginForm']/div/div[1]/div/label/input", username, by=XPATH)
    # entering the password in password field
    waiter.find_write(driver, "//*[@id='loginForm']/div/div[2]/div/label/input", password, by=XPATH)
    # clicking on the login button
    waiter.find_element(driver, "//*[@id='loginForm']/div/div[3]", by=XPATH).click()
    wait = WebDriverWait(driver, 10)
    message_btn = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[text()='Not now']")))
    return 

In [29]:
from tqdm import tqdm

def scrape_again(driver,acc):
    
    try:
        driver.get(f"https://www.instagram.com/{acc}/")
        try:
            wait_until = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME , "h2")))
        except:
            wait_until = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME , "h1")))

        html_source = driver.page_source
        sourcedata = html_source.encode('utf-8')
        soup = BeautifulSoup( sourcedata , 'html.parser')

        try:
            screename = soup.find('h2',{"class":"_aacl _aacs _aact _aacx _aada"}).text
        except:
            screename = 'NULL'

        po_fo_fol = soup.find_all('span',{"class":"_ac2a"})
        posts_count = po_fo_fol[0].text
        followers_count = po_fo_fol[1].text
        following_count = po_fo_fol[2].text

        try:
            name = soup.find('span',{"class":"_aacl _aacp _aacw _aacx _aad7 _aade"}).text
        except:
            name = 'NULL'

        is_verified = False
        if soup.find('span',{"class":"_act0 _a9_u _9ys7"}):
            is_verified = True

        bio = 'NULL'
        try:
            if soup.find_all('div',{"class":"_aacl _aacp _aacu _aacx _aad6 _aade"})[3]:
                bio = soup.find_all('div',{"class":"_aacl _aacp _aacu _aacx _aad6 _aade"})[3].text
        except:
            bio = 'NULL'

        bio_url = 'NULL'
        if soup.find('div',{"class":"_aacl _aacp _aacw _aacz _aada _aade"}):
            bio_url = soup.find('div',{"class":"_aacl _aacp _aacw _aacz _aada _aade"}).text


        is_bussiness_acc = False;bussiness_sector= 'NULL'
        if soup.find('div',{"class":"_aacl _aacp _aacu _aacy _aad6 _aade"}):
            is_bussiness_acc = True
            bussiness_sector = soup.find('div',{"class":"_aacl _aacp _aacu _aacy _aad6 _aade"}).text

        is_private = True
        if soup.find('span',{"class":"_ac-g"}):
            is_private = False

        user_id = soup.find('script',{'type':"application/json"}).text
        user_id = user_id[46:-17]

        data={
            'user id' : user_id,
            'name':name,
            'screen-name':screename,
            'bio' : bio,
            'bio url' : bio_url,
            'followers' : followers_count,
            'following' : following_count,
            'posts' : posts_count,
            'is verified':is_verified,
            'is private' : is_private,
            'is bussiness account' : is_bussiness_acc,
            'bussiness sector' : bussiness_sector

        }
        result_dict.update({acc:data})
    except:
        pass

In [30]:
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.proxy import Proxy, ProxyType
# import pickle


# options = Options()

# # options.headless = True
# options.add_argument("--incognito")
# driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install())) 

# login(driver)

# pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))

In [31]:
alread_scraped_df = pd.read_excel("sofia_davis156's_insta_followers.xlsx")
accs = alread_scraped_df['followers_usernames']

result_dict = dict()

options = Options()

options.headless = True
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install())) 
driver.get(f"https://www.instagram.com/")
cookies = pickle.load(open("instacookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

scrape_again(driver,accs)


[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Driver [C:\Users\sasiv\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [25:26<00:00,  3.75s/it]


In [32]:
df

user id             name     screen-name  \
0    7152384252551458534       Johel Baez      baez_johel   
1    7152384269852213272           Ki Sha    ki.sha_black   
2    7152384283070489227             NULL            NULL   
3    7152384295381846254  Luchencko Matta  luchenckomatta   
4    7152384308676859797         HøÜš Sam   hedjazhoussam   
..                   ...              ...             ...   
393  7152390571752063112     doitttrading     doittrading   
394  7152390587386233665       Queen nina    queennina651   
395  7152390601640352720  Sukhrob Murodov      sukhrob_mb   
396  7152390617465332488             NULL            NULL   
397  7152390669288972567    Müstafa  Türķ    must_afa7741   

                                                   bio  \
0    Libriano ☆Traders, Empresario 📈📊24 inviernos E...   
1                               CE0 kyristalMayLow key   
2                                          @fenerbahce   
3    Mexicano, de agave y tabaco.                  ...   
4    الحمد للهÉtudiant à l'université de Constantin...   
..                                                 ...   
393  Find the newest and most innovative EAs and in...   
394                                               NULL   
395                                               NULL   
396                                               NULL   
397                                               NULL   

                                  bio url followers following posts  \
0                                    NULL       283     2,114    25   
1                                    NULL       355     1,244    38   
2                                    NULL       531     1,057     0   
3                                    NULL       705       956   572   
4                                    NULL       202     3,720     2   
..                                    ...       ...       ...   ...   
393  www.mql5.com/en/market/product/86581       824     3,787   121   
394                                  NULL        55       104     1   
395                                  NULL       327       657    47   
396                                  NULL        69       234     0   
397                                  NULL        36       133     0   

    is verified is private is bussiness account bussiness sector  
0         False      False                 True            Gamer  
1         False       True                False             NULL  
2         False       True                False             NULL  
3         False       True                False             NULL  
4         False       True                False             NULL  
..          ...        ...                  ...              ...  
393       False      False                False             NULL  
394       False      False                False             NULL  
395       False      False                False             NULL  
396       False      False                False             NULL  
397       False      False                False             NULL  

[398 rows x 12 columns]

In [27]:
len(result_dict)

176

In [91]:
sourcedata = html_source.encode('utf-8')
soup = BeautifulSoup( sourcedata , 'html.parser')

In [92]:
screename = soup.find('h2',{"class":"_aacl _aacs _aact _aacx _aada"}).text
screename

'sasivatsal'

In [93]:
po_fo_fol = soup.find_all('span',{"class":"_ac2a"})
posts_count = po_fo_fol[0].text
followers_count = po_fo_fol[1].text
following_count = po_fo_fol[2].text

print(posts_count,followers_count,following_count)

7 433 455


In [94]:
name = soup.find('span',{"class":"_aacl _aacp _aacw _aacx _aad7 _aade"}).text
name

'Sasi Vatsal'

In [95]:
is_verified = False
if soup.find('span',{"class":"_act0 _a9_u _9ys7"}):
    is_verified = True
is_verified

False

In [96]:
bio = 'NULL'
if soup.find_all('div',{"class":"_aacl _aacp _aacu _aacx _aad6 _aade"})[3]:
    bio = soup.find_all('div',{"class":"_aacl _aacp _aacu _aacx _aad6 _aade"})[3].text
bio

'𝚊𝚒𝚝𝚎e𝚗𝚕𝚊𝚍|𝙰𝚜𝚝𝚛𝚘𝚙𝚑𝚒𝚕𝚎🔭🌘𝙾𝚗 𝚟𝚘𝚢𝚊𝚐𝚎 𝚒𝚗 𝚜𝚎𝚊𝚛𝚌𝚑 𝚘𝚏 𝚊𝚕𝚙𝚑𝚊𝚌𝚎𝚗𝚝𝚞𝚛𝚊𝚒𝙱 🚀𝙿𝚊𝚛𝚝 𝚝𝚒𝚖𝚎 𝚜𝚝𝚞𝚍𝚎𝚗𝚝🧖|𝙵𝚞𝚕𝚕𝚝𝚒𝚖𝚎 𝙽𝚎𝚝𝚏𝚕𝚒𝚡 𝚋𝚒𝚗𝚐𝚎𝚛🧑\u200d💻𝙸 turn coffee into code 🛐'

In [97]:
bio_url = 'NULL'
if soup.find('div',{"class":"_aacl _aacp _aacw _aacz _aada _aade"}):
    bio_url = soup.find('div',{"class":"_aacl _aacp _aacw _aacz _aada _aade"}).text
bio_url

'NULL'

In [98]:
is_bussiness_acc = False;bussiness_sector=None
if soup.find('div',{"class":"_aacl _aacp _aacu _aacy _aad6 _aade"}):
    is_bussiness_acc = True
    bussiness_sector = soup.find('div',{"class":"_aacl _aacp _aacu _aacy _aad6 _aade"}).text
if bussiness_sector:
    print(is_bussiness_acc)
    print(bussiness_sector)
else:
    print(is_bussiness_acc)

False


In [112]:
is_private = False
if soup.find('span',{"class":"_ac-g"}):
    is_private = True
is_private

True

In [123]:
ds = soup.find('script',{'type':"application/json"}).text
ds[46:-17]

'7152163973320430513'